## Chapter 6: Solution of Linear and non-linear equations

A common task in computational physics involves solving systems of equations. We will use the example from the text of an equation
$$
2w+x+4y+z=6 \\
3w+4x-y-z=3 \\
w-4x+y+5z=9 \\
2w-2x+y+3z=7
$$
which we may also write either as
$$
\begin{pmatrix}
2 & 1 & 4 & 1\\
3 & 4 & -1 & -1\\
1 & -4 & 1 & 5\\
2 & -2 & 1 & 3
\end{pmatrix}
\begin{pmatrix}
w\\ x\\ y\\ z
\end{pmatrix}
=
\begin{pmatrix}
-4\\ 3\\ 9\\ 7
\end{pmatrix}
$$
or **Ax=v**.
One way to solve this (not the best though) is to compute $A^{-1}$ and multiply both sides. Computing inverse matrices can be difficult and there are faster ways. One of the most common ways is *Gaussian elimination*. There are two rules for how we may change are system of equations which will not affect the solution, **x**. 
1. We can multiply a row of **A** by a constant as long as we also multiply the same row in **v** by the same constant.
2. We can add or subtract from any row of **A**, a multiple of any other row, as long as we do the same for **v**.

In the matrix above we could divide the first row (**A** and **v**) by two and subtract 3 times the first from the second row to obtain
$$
\begin{pmatrix}
1 & 0.5 & 2 & 0.5\\
0 & 2.5 & -7 & -2.5\\
1 & -4 & 1 & 5\\
2 & -2 & 1 & 3
\end{pmatrix}
\begin{pmatrix}
w\\ x\\ y\\ z
\end{pmatrix}
=
\begin{pmatrix}
-4\\ 3\\ 9\\ 7
\end{pmatrix}
$$
We won't go through all the steps (see the book for this) but you eventually arrive at an upper triangular matrix with 1's on the diagonal
$$
\begin{pmatrix}
1 & 0.5 & 2 & 0.5\\
0 & 1 & -2.8 & -1\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
w\\ x\\ y\\ z
\end{pmatrix}
=
\begin{pmatrix}
-2\\ 3.6\\ -2\\ 1
\end{pmatrix}
$$
The reason I don't want to type all the steps is because it is very tedious. In fact, the perfect job for a computer! But it could be illustrative to review this. 

###@ Backsupstitution
We see the last equation tells us that $z=1$. Once we know that, we substitute into the second equation from the bottom and solve for y. We then substitute z and y into the third from bottom to solve for x. Then repeat for w. 

If you were to work through the math you would find
$$
w=2, \quad x=-1, \quad y=-2, \quad \text{and} \quad z=1.
$$

## Exercise:
For some practice, in the next cell enter the code from Example 6.1 and solve this system of equations. (You can also copy and paste code from the textbook website, but take time to think about what the code is doing.) Make note of the following features that the textbook highlights. 
1. We store matrices and vectors as arrays in Python.
2. We use Python's ability to perform operations on entire rows at once. the operations are performed on all the elements of that row (array). This makes the code much simpler and faster in Python.
## ANSWER [Enter code and run. Did you get (w,x,y,z)=(2, -1, -2, 1)?]

In [9]:
import numpy as np
A = np.array([[2, 1, 4, 1],
              [3, 4, -1, -1],
              [1, -4, 1, 5],
              [2, -2, 1, 3]], float)
v = np.array([-4, 3, 9, 7], float)
N = len(v)

#Elimination step
for m in range(N):
    
    #Dividing bydiagonal element
    div = A[m, m]
    A[m,:] /= div
    v[m] /= div
    
    #Subtracting from lower rows
    for i in range(m+1, N):
        mult = A[i, m]
        A[i,:] -= mult*A[m,:]
        v[i] -= mult*v[m]
    
#Backsubstitution
x = np.empty(N, float)
for m in range(N-1, -1, -1):
    x[m] = v[m]
    for i in range(m+1,N):
        x[m] -= A[m, i]*x[i]
    
print(x)

[ 2. -1. -2.  1.]


### Pivoting

An issue to be aware of when using the Gaussian elimination is a case where a zero appears on the diagonal for one row. Consider a matrix where the value, $A[0,0]$, in the top left position of the matrix is zero:
$$
\begin{pmatrix}
0 & 1 & 4 & 1\\
3 & 4 & -1 & -1\\
1 & -4 & 1 & 5\\
2 & -2 & 1 & 3
\end{pmatrix}
\begin{pmatrix}
w\\ x\\ y\\ z
\end{pmatrix}
=
\begin{pmatrix}
-4\\ 3\\ 9\\ 7
\end{pmatrix}
$$.

### Question: Follow the code in the above example. Which line of code causes a problem if A[0,0]=0?
## ANSWER: Lines 14 and 15 become problemmatic since they would be dividing by 0. This will raise an error and the Gaussian elimination won't work.

Well that is certainly a problem. It turns out there is a simple solution. Our system of equations does not care about the order of the equations. We can interchange two rows in our matrix on the left, as long as we also swap the same rows in the vector **v** on the right side. An improvement to our algorithm will catch these cases and swap them when they occur. The general procedure is to swap the row that has this issue (1 in this case) with the equation that has the largest coefficient in that column. The second row includes 3 in the first column. 3 is the largest value in that column vector. So we can try to interchange the first and second rows: 
$$
\begin{pmatrix}
3 & 4 & -1 & -1\\
0 & 1 & 4 & 1\\
1 & -4 & 1 & 5\\
2 & -2 & 1 & 3
\end{pmatrix}
\begin{pmatrix}
w\\ x\\ y\\ z
\end{pmatrix}
=
\begin{pmatrix}
3\\ -4\\ 9\\ 7
\end{pmatrix}
$$.
Now we could proceed with our Gaussian elimination algorithm.

### Exercise 6.1: A circuit of resistors
Let's apply the program above to determine the voltages in the circuit given in the problem.  From Kirchhoff's laws we know that the sum of currents through each junction must add to zero (i=voltage difference/resistance as demonstrated in the equation for the point $V_{1}$. In part a, you will end up with four equations and four unknown voltages. The coefficients form your matrix A, the voltages your vector x, and the vector on the right side will be your know voltages: $V_{+}$ and 0.

In [12]:
#System that represents circuit of resistors
A = np.array([[4, -1, -1, -1],
             [-1, 3, 0, -1],
             [-4, 0, 3, -1],
             [0, -1, -1, 3]], float)
v = np.array([1, 0, 1, 0], float)

N = len(v)

#Elimination step
for m in range(N):
    
    #Dividing bydiagonal element
    div = A[m, m]
    A[m,:] /= div
    v[m] /= div
    
    #Subtracting from lower rows
    for i in range(m+1, N):
        mult = A[i, m]
        A[i,:] -= mult*A[m,:]
        v[i] -= mult*v[m]
    
#Backsubstitution
x = np.empty(N, float)
for m in range(N-1, -1, -1):
    x[m] = v[m]
    for i in range(m+1,N):
        x[m] -= A[m, i]*x[i]
#The respective voltages
print(x)

[1.375      0.83333333 2.54166667 1.125     ]


### numpy.linalg functions
Because these calculations are so common in physics the methods are coded for easy use in packages. This is the case in most programming languages, not just Python. For example, Press, et. al. is a famous Numerical Recipes in C. The important thing is to have some understanding of the general methods that are used. Learning some limitations and issues that may arise in real applications can help you spot problems or avoid them in the first place. 

In [13]:
from numpy.linalg import solve
y = solve(A,v)
print(y)

[1.375      0.83333333 2.54166667 1.125     ]


### Question: How does this result compare to the method above?
### ANSWER: The answers match!


### LU decomposition.

In many cases the matrix **A** on the left side of our equation **Ax**=**v** is the same. The solutions we seek are different only because of different values of the vector **v** on the right side of the equation. In this case, the process of Gaussian elimination (the multiplication/division and adding/subtracting of rows) will be the same on the left every single time! In *LU decomposition* we can keep track of these steps and avoid repeating the entire process over and over. (The book desribes the process in more detail, but refers the reader to www.scipy.org for additional information on LU decomposition using the numpy.linalg package. There are pre-built functions to do this if needed. We won't need them.)

Essentially, you can systematically perform Gaussian elimination and store the steps in N matrices and N inverse matricies. There is one for each column vector in the matrix. **L** is a **l**ower triangular matrix, **U** is an **u**pper triangular matrix which is why it is called **LU** decomposition.
$$
LU = A  \\
L = L_{0}^{-1}L_{1}^{-1}L_{2}^{-1}L_{3}^{-1}  \\
U = L_{3}L_{2}L_{1}L_{0} \\
LUx = v
$$

What you should recognize is we had a number of *for loops* in our code from Example 6.1. As we learned a few weeks ago, nested for loops can may take a long time to process. Imagine if we had a very large matrix. Using *LU decomposition* we can do those for loops one time and store the results in the **L** and **U** matrices. For all subsequent cases we may reuse **L** and **U** with a different **v**. We have largely reduced the problem to matrix multiplication.

This is a good technique to be aware of if you encounter large matrices in your work.

### Inverse matrices

In physics we often do not need to compute inverse matrices. The standard mathematical procedure computing cofactors and determinants may have issues and the same procedures we discussed above can be employed. However, there are pre-built functions to invert matrices included in the numpy.linalg package.

## Exercise: 
In the next cell, copy your definition of matrix A from Example 6.1 coded above. Compute $A^{-1}$.
## ANSWER

In [16]:
from numpy.linalg import inv

# Copy your definitions of A from the Example 6.1 coded above.
A = np.array([[2, 1, 4, 1],
              [3, 4, -1, -1],
              [1, -4, 1, 5],
              [2, -2, 1, 3]], float)

# Here is how to compute the inverse matrix of A.
X = inv(A)
print(X)

[[-1.17647059e-01 -5.88235294e-02 -5.88235294e-01  1.00000000e+00]
 [ 1.76470588e-01  3.38235294e-01  6.32352941e-01 -1.00000000e+00]
 [ 2.35294118e-01 -1.32352941e-01 -7.35294118e-02  2.61228947e-17]
 [ 1.17647059e-01  3.08823529e-01  8.38235294e-01 -1.00000000e+00]]


## Eigenvectors and Eigenvalues

There are a number of approaches to matrix or linear algebra calculations that are common in physics. Similar to LU decomposition, these approaches record steps of the solutions in two matrices that can be used later. The matrices have properties that are advantagous. *QR decomposition* is the most common approach to solving eigenvalue and eigen vector problems. This approach will work for many common problems in physics where we encounter real, symmetric, square matricies (and also complex valued but Hermitian matricies). **Q** is chosen as an orthogonal matrix. It's column vectors are orthogonal. And **R** is chosen as an upper triangular matrix. It is an iterative process. The goal is to arrive at a diagonal matrix so all off diagonal entries are zero (or within some chosen small value $\epsilon$) and each column is an eigenvector corresponding to one eigenvalue $\lambda$. The eigenvalues are on the diagonal.

Let us define the eigenvalue problem:
$$
Av_{i} = \lambda v_{i}
$$
where **$v_{i}$** are orthogonal vectors. Or, we can collect these as columns of an orthogonal matrix **V** with an eigenvalue $\lambda$
$$
AV = VD.
$$
First write,
$$
A = Q_{1}R_{1}
$$
Multiply by on the right
$$
Q_{1}^{T}A = Q_{1}^{T}Q_{1}R_{1} = R_{1}
$$
Now 
$$
A_{1} = R_{1}Q_{1}
$$
Substitute $R_{1}$ here so that
$$
A_{1} = Q_{1}^{T}AQ_{1}
$$
The process can be repeated
$$
A_{2} = Q_{2}^{T} A_{1} Q_{2} = Q_{2}^{T}Q_{1}^{T} A Q_{1}Q_{2} \\
A_{3} = Q_{3}^{T} A_{2} Q_{3} = Q_{3}^{T}Q_{2}^{T}Q_{1}^{T} A Q_{1}Q_{2}Q_{3}
$$
The process is repeated *k* times until these matrices have close to zero off-diagonal elements.
$$
A_{k} = Q_{k}^{T}...Q_{2}^{T}Q_{1}^{T} \ A \ Q_{1}Q_{2}...Q_{k}
$$
We can write these produces of matrices **Q** which are orthogonal, as V
$$
A_{k} = V^{T}AV
$$
The process is iterative until a desired, specified accuracy is obtained. The textook outlines the process
1. Create an NxN matrix V. Initially it is the identity matrix, **I**. But it will be transformed to a matrix of eigenvectors. Also, choose $\epsilon$, which is how close the off-diagonal elements must be.
2. Calculate the *QR decomposition* **A**=**QR**.
3. Update **A** to the new value **A**=**RQ** (order is switched).
4. Multiply *V** on the right by **Q**.
5. Check the off-diagonal elements of A. If they are all less than $\epsilon$ stop, else repeat starting with step 2.

It is rare to code this, but is a very informative exercise, if you will be using this often. In fact, Exercise 6.8 guides you through writing a program. We will use pre-coded algorithms to explore important features to be aware of.

First, lets look at a simple implementation with the *eigh* function. It returns vector x and matrix V.

In [17]:
from numpy import array
from numpy.linalg import eigh, eigvalsh
A = array([[ 1, 2 ], [ 2, 1 ]], float)
x,V = eigh(A)
print(x)
print(V)

[-1.  3.]
[[-0.70710678  0.70710678]
 [ 0.70710678  0.70710678]]


Or if we only want the eigenvalues use this function

In [45]:
A = array([[ 1, 2 ], [ 2, 1 ]], float)
x = eigvalsh(A)
print(x)

[-1.  3.]


The textbook claims that this will run without an error if the matrix is not square. It will simply assume the upper triangular elements are the same as the lower triangular elements
$$
a_{i,j} = a_{j,i}
$$
I do not see this so perhaps the algorithms in numpy.linalg have been *upgraded*. However, this is a potential issue to be aware of in case you run across it.
    
A last note, that there are two other functions *eig* and *eigvals* that work for non-semmetric eigenvalue problems but these are rare in physics.

## Nonlinear Equations

Often in physics we encounter equations that are not linear or a linear set of equations. Other techniques can be applied. The *relaxation method*  is simple but extremely useful in many cases. Take an equation such as
$$
f(x) = x-2+e^{-x} = 0
$$
We want the solutions when $f(x)=0$. This will happen when
$$
x = 2-e^{-x}
$$
    Just guess a value, such as $x=1$ on the right and compute $x$ on the left. Use this new value of x on the right, and compute another value of $x$ on the left. You can check if the value you just input becomes very close to the value you get out. If so, you have found your solution!

## Exercise: 
In the next cell, code a simple program to loop over this equation $x = 2-e^{-x}$ 10 times and print the value of x. What number does it converge to and how many decimal places?
## ANSWER (below)

In [23]:
x = 0.5
for i in range(10):
    x = 2-np.exp(-x)
    print(x)

1.3934693402873666
1.751787325113973
1.8265363696849992
1.8390298555971287
1.841028423293983
1.8413458214753817
1.8413961700324237
1.8414041553053788
1.8414054217314317
1.8414056225796105


## Exercise:
Repeat with the function $x = e^{1-x^{2}}$ for 10 iterations starting with an initial guess of $x=0.5$. Put 1 on each side of this equation. Clearly x=1 is a solution. But does the algorithm converge?
## ANSWER: This algorithm doesn't converge, it oscillates between $e$ and 0


In [24]:
x = 0.5
for i in range(10):
    x = np.exp(1-x**2)
    print(x)

2.117000016612675
0.030755419069985038
2.715711832754083
0.0017034651847384463
2.71827394057758
0.001679913095081425
2.7182741571849562
0.0016799111168229455
2.7182741572030236
0.0016799111166579386


## Exercise:
Now rewrite this as $x = \sqrt{1-log{x}}$ and retry with initial guess of x=0.5. Does this same equation, rewritten slightly, converge?
## ANSWER: Yes, this converges to 1.

In [26]:
x = 0.5
for i in range(10):
    x = np.sqrt(1-np.log(x))
    print(x)

1.3012098910475378
0.8583154914892762
1.0736775779454883
0.9637999044091371
1.0182689104343374
0.990906635925747
1.004557096969838
0.997724037576543
1.0011386299421705
0.9994308469350205


It turns out there is a condition
$$
\left| f^{'}(x*) \right| < 1
$$
If the first deriviative is not less than one at the solution x*, the relaxation method does not work. You can sometimes find an alternative way to right the equation that will work. This is why the method did not work for $x = \exp(1-x^2)$ but did for $x = \sqrt{1-\log{x}}$.
## Exercise
Verify this. Take the derivative, $|f^{'}(x)|$, of the right-hand side of these equations. That is do this for each of these expressions: $\exp(1-x**2)$ and $\sqrt{1-\log{x}}$. 
## ANSWER
$$
\frac{d}{dx}e^{1-x^2}=-2xe^{1-x^2}, |f'(1)|=2>1 \\
\frac{d}{dx}\sqrt{1-\log{x}}=\frac{1}{2x\ln{10}\sqrt{1-\log{x}}}, |f'(1)|=\frac{1}{2\ln(10)}<1
$$